In [1]:
import datetime
import os

import numpy as np
import tensorflow as tf

In [2]:
print(f"Tensorflow Version: {tf.__version__}")
print(f"Eager mode: {tf.executing_eagerly()}")
print(f"{tf.config.list_physical_devices('GPU')}")

Tensorflow Version: 2.1.0
Eager mode: True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [4]:
train_image = tf.cast(tf.expand_dims(train_image, -1)/255, tf.float32)
train_label = tf.cast(train_label, tf.int64)
test_image = tf.cast(tf.expand_dims(test_image, -1)/255, tf.float32)
test_label = tf.cast(test_label, tf.int64)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label))
train_dataset = train_dataset.repeat().shuffle(60000).batch(128)
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [6]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_label))
test_dataset = test_dataset.repeat(1).batch(128)
test_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dense(10, activation='softmax'),
])

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [9]:
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%T%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
# Customized feature in tensorflow board
file_writer = tf.summary.create_file_writer(log_dir+'/lr')
file_writer.set_as_default()

def lr_sche(epoch):
    lr = 0.2
    if epoch > 5:
        lr = 0.02
    if epoch > 10:
        lr = 0.01
    if epoch > 20:
        lr = 0.005
    tf.summary.scalar('learning_rate', data=lr, step=epoch)
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [11]:
model.fit(
    train_dataset,
    epochs=25,
    steps_per_epoch=60000//128,
    validation_data=test_dataset,
    validation_steps=10000//128,
    callbacks=[tensorboard_callback, lr_callback],
)

Train for 468 steps, validate for 78 steps
Epoch 1/25
468/468 [==============================] - 3s 5ms/step - loss: 2.3470 - accuracy: 0.1036 - val_loss: 2.3186 - val_accuracy: 0.1010
Epoch 2/25
468/468 [==============================] - 1s 3ms/step - loss: 2.3125 - accuracy: 0.1048 - val_loss: 2.3069 - val_accuracy: 0.1029
Epoch 3/25
468/468 [==============================] - 2s 3ms/step - loss: 2.3131 - accuracy: 0.1027 - val_loss: 2.3126 - val_accuracy: 0.0975
Epoch 4/25
468/468 [==============================] - 1s 3ms/step - loss: 2.3115 - accuracy: 0.1034 - val_loss: 2.3088 - val_accuracy: 0.1010
Epoch 5/25
468/468 [==============================] - 2s 3ms/step - loss: 2.3132 - accuracy: 0.1040 - val_loss: 2.3136 - val_accuracy: 0.1010
Epoch 6/25
468/468 [==============================] - 1s 3ms/step - loss: 2.3133 - accuracy: 0.1020 - val_loss: 2.3118 - val_accuracy: 0.1135
Epoch 7/25
468/468 [==============================] - 2s 3ms/step - loss: 2.3025 - accuracy: 0.1100 - val

In [12]:
%load_ext tensorboard
%matplotlib inline

In [13]:
%tensorboard --logdir logs/

Reusing TensorBoard on port 6006 (pid 15555), started 2:34:12 ago. (Use '!kill 15555' to kill it.)

### End of file